In [2]:
# Import necessary libraries
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV,train_test_split ,StratifiedKFold,KFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,ExtraTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier,BaggingRegressor,RandomForestClassifier,AdaBoostClassifier,StackingClassifier
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os

warnings.simplefilter('ignore')

os.chdir('D:\Datasets')

hr = pd.read_csv('Hr_comma_sep.csv')
hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,support,low
14991,0.37,0.48,2,160,3,0,1,0,support,low
14992,0.37,0.53,2,143,3,0,1,0,support,low
14993,0.11,0.96,6,280,4,0,1,0,support,low


In [3]:
dum_hr = pd.get_dummies(hr, drop_first=True)

# Separate features (X) and target variable (y)
x = dum_hr.drop(['left'], axis=1)
y = dum_hr['left']
dum_hr

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department_RandD,Department_accounting,Department_hr,Department_management,Department_marketing,Department_product_mng,Department_sales,Department_support,Department_technical,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,False,False,False,False,False,False,True,False,False,True,False
1,0.80,0.86,5,262,6,0,1,0,False,False,False,False,False,False,True,False,False,False,True
2,0.10,0.77,6,247,4,0,1,0,False,False,False,False,False,False,True,False,False,True,False
3,0.92,0.85,5,259,5,0,1,0,False,False,False,False,False,False,True,False,False,True,False
4,0.89,1.00,5,224,5,0,1,0,False,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14990,0.40,0.57,2,151,3,0,1,0,False,False,False,False,False,False,False,True,False,True,False
14991,0.37,0.48,2,160,3,0,1,0,False,False,False,False,False,False,False,True,False,True,False
14992,0.37,0.53,2,143,3,0,1,0,False,False,False,False,False,False,False,True,False,True,False
14993,0.11,0.96,6,280,4,0,1,0,False,False,False,False,False,False,False,True,False,True,False


# Stacking Classifier with Hyperparameter Tuning

In [8]:
# Create instances of the base classifiers
lr = LogisticRegression()
svm = SVC(probability=True, random_state=24)
dtc = DecisionTreeClassifier(random_state=24)
rf = RandomForestClassifier(random_state=24, n_estimators=25)

# Create the Stacking Classifier
# The base classifiers are passed as a list of tuples, where the first element is the name of the classifier and the second is the classifier object
# The final_estimator is the Random Forest Classifier
# 'passthrough=True' means the input features will be passed to the final estimator
stack = StackingClassifier([('KNN', knn), ('SVM', svm), ('TREE', dtc)], final_estimator=rf, passthrough=True)

# Split the data into training and testing sets
# 'x' and 'y' are the feature and target variables, respectively
# 'test_size=0.3' means 30% of the data will be used for testing
# 'stratify=y' ensures the class distribution in the training and testing sets is the same as the original data
# 'random_state=24' sets the random seed for reproducibility
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=24)

# Fit the Stacking Classifier to the training data
stack.fit(x_train, y_train)

# Use the trained Stacking Classifier to predict the classes of the test data
y_pred = stack.predict(x_test)

# Calculate and print the accuracy score of the predictions
print("Accuracy Score:", accuracy_score(y_test, y_pred))

# Calculate and print the log loss of the predictions
y_pred_prob = stack.predict_proba(x_test)[:, 1]
print("Log Loss:", log_loss(y_test, y_pred_prob))

0.9837741720382307
0.20590088677936663


In [9]:
print(stack.get_params())

{'cv': None, 'estimators': [('KNN', KNeighborsClassifier()), ('SVM', SVC(probability=True, random_state=24)), ('TREE', DecisionTreeClassifier(random_state=24))], 'final_estimator__bootstrap': True, 'final_estimator__ccp_alpha': 0.0, 'final_estimator__class_weight': None, 'final_estimator__criterion': 'gini', 'final_estimator__max_depth': None, 'final_estimator__max_features': 'sqrt', 'final_estimator__max_leaf_nodes': None, 'final_estimator__max_samples': None, 'final_estimator__min_impurity_decrease': 0.0, 'final_estimator__min_samples_leaf': 1, 'final_estimator__min_samples_split': 2, 'final_estimator__min_weight_fraction_leaf': 0.0, 'final_estimator__n_estimators': 25, 'final_estimator__n_jobs': None, 'final_estimator__oob_score': False, 'final_estimator__random_state': 24, 'final_estimator__verbose': 0, 'final_estimator__warm_start': False, 'final_estimator': RandomForestClassifier(n_estimators=25, random_state=24), 'n_jobs': None, 'passthrough': True, 'stack_method': 'auto', 'verb

# Hyperparameter Tuning for Stacking Classifier

In [ ]:
# Create an instance of the KNN Classifier
knn = KNeighborsClassifier()

# Create a Stratified K-Fold cross-validation object
# This will split the data into 5 folds, ensuring the class distribution in each fold is the same as the original data
# 'shuffle=True' ensures the data is shuffled before splitting
# 'random_state=24' sets the random seed for reproducibility
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

# Define the hyperparameter grid to search over
# The parameters are specified for the base classifiers (KNN, SVM, Decision Tree) and the final estimator (Random Forest)
params = {
    'KNN__n_neighbors': [3, 5],
    'SVM__kernel': ['linear', 'rbf'],
    'TREE__max_depth': [None, 6],
    'final_estimator__max_features': [3, 5, 7]
}

# Create a GridSearchCV object
# This will perform a grid search to find the best combination of hyperparameters
# 'cv=kfold' uses the Stratified K-Fold cross-validation object
# 'verbose=3' sets the verbosity level to 3, which will print progress updates
# 'scoring='neg_log_loss'' sets the scoring metric to negative log loss (lower is better)
gcv = GridSearchCV(stack, param_grid=params, cv=kfold, verbose=3, scoring='neg_log_loss')

# Fit the GridSearchCV object to the data
# This will perform the grid search and find the best hyperparameters
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score (i.e., the lowest negative log loss) found during the grid search
print("Best Score:", gcv.best_score_)

# Get the best model from the GridSearchCV object
best_model = gcv.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
